In [ ]:
%pip install "autogen-core"
%pip install -U "autogen-agentchat"
%pip install "autogen-ext[openai]"

In [3]:
try:
    import nest_asyncio
    nest_asyncio.apply()
    print("Async environment configured for Jupyter.")
except ImportError:
    print("Please install nest_asyncio with `pip install nest_asyncio`")

Async environment configured for Jupyter.


In [4]:
import os
import getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"var: ")

_set_env("OPENAI_API_KEY")

In [2]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.messages import TextMessage
from autogen_core import CancellationToken
import os
import asyncio

# Simple Agent

In [3]:
import asyncio
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.agents import AssistantAgent
from autogen_core import CancellationToken
from autogen_ext.models.openai import OpenAIChatCompletionClient

async def main() -> None:
    model_client = OpenAIChatCompletionClient(model="gpt-4o", seed=42, temperature=0)

    assistant = AssistantAgent(
        name="assistant",
        system_message="You are a helpful assistant that helps students \
            understand how to build AI Agents! You always greet students \
                by saing 'Welcome to this Course about Autogen!'",
        model_client=model_client,
    )

    cancellation_token = CancellationToken()
    response = await assistant.on_messages([TextMessage(content="Hello!", source="user")], cancellation_token)
    print(response)

# This allows running async code in Jupyter

asyncio.run(main())

Response(chat_message=TextMessage(source='assistant', models_usage=RequestUsage(prompt_tokens=47, completion_tokens=21), content='Welcome to this Course about Autogen! How can I assist you today in learning about AI agents?', type='TextMessage'), inner_messages=[])


# Multimodal Agent

In [6]:
import asyncio
from pathlib import Path
from autogen_agentchat.messages import MultiModalMessage
from autogen_agentchat.agents import AssistantAgent
from autogen_core import CancellationToken, Image
from autogen_ext.models.openai import OpenAIChatCompletionClient

async def main() -> None:
    model_client = OpenAIChatCompletionClient(model="gpt-4o", seed=42, temperature=0)

    assistant = AssistantAgent(
        name="assistant",
        system_message="You are a helpful assistant.",
        model_client=model_client,
    )

    cancellation_token = CancellationToken()
    message = MultiModalMessage(
        content=["Here is an image:", Image.from_file(Path("test.png"))],
        source="user",
    )
    response = await assistant.on_messages([message], cancellation_token)
    print(response)

asyncio.run(main())

Response(chat_message=TextMessage(source='assistant', models_usage=RequestUsage(prompt_tokens=1172, completion_tokens=104), content='The image shows a code snippet in a programming environment. It appears to be Python code, importing various modules and defining an asynchronous function `main()`. The function initializes a `model_client` using `OpenAIChatCompletionClient` with parameters like `model="gpt-4o"`, `seed=42`, and `temperature`. An `AssistantAgent` is also being created with the name "assistant". The terminal at the bottom shows the directory path `~/Desktop/projects/learning/autogen-stuff`.', type='TextMessage'), inner_messages=[])


# Adding Tools

In [9]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_core import CancellationToken

async def fetch_weather(city: str) -> str:
    return f"The weather in {city} is sunny."

# Initialize the agent
model_client = OpenAIChatCompletionClient(model="gpt-4o", seed=42, temperature=0)
assistant = AssistantAgent(
    name="assistant",
    model_client=model_client,
    system_message="You return weather information given a city name using the fetch_weather tool.",
    tools=[fetch_weather], # Register the tool function
    )



output = await assistant.on_messages([TextMessage(content="What is the weather in San Francisco?", source="user")], CancellationToken())

In [17]:
for msg in output.inner_messages:
    print(msg.content)

[FunctionCall(id='call_g7pkSkwGDQxMviJLarrSclFK', arguments='{"city":"San Francisco"}', name='fetch_weather')]
[FunctionExecutionResult(content='The weather in San Francisco is sunny.', call_id='call_g7pkSkwGDQxMviJLarrSclFK')]


In [18]:
output.chat_message.content

'The weather in San Francisco is sunny.'